In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
import json

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

from RecAlg import GazeData
from RecAlg import PopulatioData
from RecAlg import SearchTrendData
from RecAlg import ExportAdvList
from util_data import *
from random import randint
# from exposure_insert import insert_exposure_data_from_dataframe

pd.set_option('future.no_silent_downcasting', True)

In [3]:
# MySQL 연결 정보 설정
with open(f'SQL_parameter.json', 'r') as file:
    par = json.load(file)

host = par['host']
port = par['port']
username = par['username']
password = par['password']
database = par['database']

In [4]:
# 비밀번호 URL 인코딩
encoded_password = urllib.parse.quote_plus(password)

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{username}:{encoded_password}@{host}:{port}/{database}")
ad_gaze, ad_info, ad_target, ad_images, frames, categories = import_data(engine)

In [5]:
# 지역 선택
target = '군자역' # [1:삼각지역, 2:군자역, 3:회기역, 4:'용산역]
loc_num = 2

In [6]:
# 응시횟수 기반 광고 순서 결정
df1 = GazeData.score_temp(ad_info, loc_num, ad_target)
ad_gaze = GazeData.gaze_temp(ad_gaze, ad_info, frames)
df_target = GazeData.calculate_score(ad_gaze, df1)
keywords = list(df1['name'])
df_gazed = GazeData.generate_add_actual(ad_gaze, keywords)
gazed_df = GazeData.ranked_add_actual(df_gazed)

In [12]:
pop = PopulatioData.import_data(engine, loc_num)
pop, new_columns = PopulatioData.calculate_columns(pop)

In [ ]:
# 네이버 실시간 검색어 트랜드 -> ture에서만 제대로 작동동
keywords = list(gazed_df.index)
Search_option = False
if Search_option == True:
    naver_df = SearchTrendData.get_final_df(keywords)
    naver_df.to_csv('SearchTrend.csv')
else:
    naver_df = pd.read_csv('SearchTrend.csv', index_col=0)

In [19]:
st_date = '2025-01-08'
final_exposure = export_exposure_using_lstm(gazed_df, naver_df, pop, st_date)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step


d:\backend_test\bigprojectbackend\process\AdvExposureGenerator\util_data.py:208: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_exposure = pd.concat([final_exposure, new_rows.iloc[:ad_num]], ignore_index=True)


In [20]:
final_exposure = export_exposure_only_data(gazed_df, naver_df, pop, st_date)

d:\backend_test\bigprojectbackend\process\AdvExposureGenerator\util_data.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_exposure = pd.concat([final_exposure, new_rows.iloc[:ad_num]], ignore_index=True)


In [21]:
final_exposure

,name,time
0,에듀윌,2025-01-08 09:00:00
1,광고,2025-01-08 10:00:00
2,서울퍼시픽 안과의원,2025-01-08 11:00:00
3,에듀윌 토익,2025-01-08 12:00:00
4,HDC 현대산업개발,2025-01-08 13:00:00
5,뮤지컬 웃는남자,2025-01-08 14:00:00
6,서울 저스트 치과의원,2025-01-08 15:00:00
7,밀리의 서재,2025-01-08 16:00:00
8,탑마루,2025-01-08 17:00:00
9,에듀윌,2025-01-08 18:00:00


In [22]:
import pymysql
from datetime import datetime, timedelta

def insert_exposure_data_from_dataframe(df):
    # MySQL 데이터베이스 연결
    connection = pymysql.connect(
        host=par['host'],
        user=par['username'],
        password=par['password'],
        database=par['database']
    )

    try:
        with connection.cursor() as cursor:
            for _, row in df.iterrows():
                cursor.execute(
                    "SELECT ad_id FROM ad_info WHERE title = %s",
                    (row['name'],)
                )
                result = cursor.fetchone()
                                
                if result:
                    ad_id = result[0]
                    # exposure 테이블에 데이터 삽입
                    start_time = datetime.strptime(row['time'], '%Y-%m-%d %H:%M:%S')
                    end_time = start_time + timedelta(hours=1)
                    
                    cursor.execute(
                        """
                        INSERT INTO exposure (start_time, end_time, ad_id, region_id)
                        VALUES (%s, %s, %s, %s)
                        """,
                        (
                            start_time,
                            end_time,
                            ad_id,
                            2  # region_id는 2로 고정
                        )
                    )

            # 변경사항 커밋
            connection.commit()
            print("Exposure data inserted successfully!")

    except Exception as e:
        print(f"Error occurred: {e}")
        connection.rollback()

    finally:
        connection.close()

# 데이터프레임이 df라는 변수로 이미 존재한다고 가정
# 실행
if __name__ == "__main__":
    insert_exposure_data_from_dataframe(final_exposure)

Error occurred: strptime() argument 1 must be str, not Timestamp


In [ ]:
# # 결과 출력
# display(final_exposure)

# # 데이터베이스에 데이터 삽입
# insert_exposure_data_from_dataframe(final_exposure)